# King-Rook vs King-Pawn Chess Problem
## Predicting the winner of this classic final chess game

This is an endgame in a chess game where only the white king and rook are left, and the black king and pawn, where the pawn is in position a7. In this case, it's white's turn to move. So, the problem is to predict whether the white side will win or not, based on what position each piece is.

<p align="center">
<img src="img/rook-endgames.png">
</p>

# 0.0. Imports 

In [58]:
import pandas as pd
import numpy  as np

from sklearn.preprocessing         import LabelEncoder
from sklearn.model_selection       import train_test_split
from sklearn.model_selection       import cross_val_score
from sklearn.svm                   import SVC
from sklearn.neural_network        import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics               import accuracy_score

import random
import warnings
warnings.filterwarnings( "ignore" )

## 0.1. Helper Functions

In [63]:
# Support Vector Machine
def svm_model( X_train, X_test, y_train, y_test, cv, MAX_EVAL, param ):
    
    final_result = {
        'Regularization': [],
        'Gamma': [],
        'Tol': [],
        'Max_iter': [],
        'Hold_out': [],
        'Cross_validation': []
    }
    
    for i in range( MAX_EVAL ):
        
        # choose values for parameters randomly
        hp = { k: random.sample( v, 1 )[0] for k, v in param.items() }
        final_result['Regularization'].append( hp['C'] )
        final_result['Gamma'].append( hp['gamma'] )
        final_result['Tol'].append( hp['tol'] )
        final_result['Max_iter'].append( hp['max_iter'] )
        
        # model
        model_svm = SVC( C = hp['C'],
                         gamma = hp['gamma'],
                         tol = hp['tol'],
                         max_iter = hp['max_iter'],
                         random_state = 33)
        
        # fit and training
        y_pred = model_svm.fit( X_train, y_train ).predict( X_test )
        
        # performance - hold-out
        result = accuracy_score( y_test, y_pred )
        
        # performance - cross validation
        result_cv = cross_val_score( model_svm, X_train, y_train, cv=cv )
        
        final_result['Hold_out'].append( format( result, ".2f" ) )
        final_result['Cross_validation'].append( format( np.mean( result_cv ), ".2f" ) )
        
    return pd.DataFrame( final_result ) 

# Multilayer Perceptron
def mlp_model( X_train, X_test, y_train, y_test, cv, MAX_EVAL, param ):
    
    final_result = {
        'Hidden_layer_sizes': [],
        'Activation': [],
        'Solver': [],
        'Hold_out': [],
        'Cross_validation': []
    }
    
    for i in range( MAX_EVAL ):
        
        # choose values for parameters randomly
        hp = { k: random.sample( v, 1 )[0] for k, v in param.items() }
        final_result['Hidden_layer_sizes'].append( hp['hls'] )
        final_result['Activation'].append( hp['activation'] )
        final_result['Solver'].append( hp['solver'])
        
        # model
        model_mlp = MLPClassifier( hidden_layer_sizes = hp['hls'],
                                   activation = hp['activation'],
                                   solver = hp['solver'],
                                   random_state = 33 )
        
        # fit and training
        y_pred = model_mlp.fit( X_train, y_train ).predict( X_test )
        
        # performance - hold-out
        result = accuracy_score( y_test, y_pred )
        
        # performance - cross validation
        result_cv = cross_val_score( model_mlp, X_train, y_train, cv=cv )
        
        final_result['Hold_out'].append( format( result, ".2f" ) )
        final_result['Cross_validation'].append( format ( np.mean( result_cv ), ".2f" ) )
        
    return pd.DataFrame( final_result ) 

# Linear Discriminant Analysis
def lda_model( X_train, X_test, y_train, y_test, cv, MAX_EVAL, param ):
    
    final_result = {
        'Solver': [],
        'Hold_out': [],
        'Cross_validation': []
    }
    
    for i in range( MAX_EVAL ):
        
        # choose values for parameters randomly
        hp = { k: random.sample( v, 1 )[0] for k, v in param.items() }
        final_result['Solver'].append( hp['solver'] )
        
        # model
        model_lda = LinearDiscriminantAnalysis( solver = hp['solver'] )
        
        # fit and training
        y_pred = model_lda.fit( X_train, y_train ).predict( X_test )
        
        # performance - hold-out
        result = accuracy_score( y_test, y_pred )
        
        # performance - cross validation
        result_cv = cross_val_score( model_lda, X_train, y_train, cv=cv )
        
        final_result['Hold_out'].append( format( result, ".2f" ) )
        final_result['Cross_validation'].append( format ( np.mean( result_cv ), ".2f" ) )
        
    return pd.DataFrame( final_result ) 

## 0.2. Loading Data

In [27]:
df_raw = pd.read_csv( 'datasets/kr-vs-kp.data' )

# 1.0. Data Description

In [30]:
# Shuffling the dataset
df = df_raw.sample( frac = 1 )
df.head()

,f,f.1,f.2,f.3,f.4,f.5,f.6,f.7,f.8,f.9,...,f.23,f.24,f.25,f.26,f.27,f.28,t.2,t.3,n.1,won
94,f,f,f,f,f,f,t,f,f,f,...,f,f,f,f,f,f,t,f,n,won
2040,f,f,f,f,f,t,t,f,f,t,...,f,f,f,f,f,t,t,t,n,won
1937,f,f,f,t,t,t,f,f,f,f,...,f,f,f,f,f,f,f,t,t,won
2615,f,t,f,t,f,t,f,f,f,t,...,f,f,f,f,f,t,t,t,n,nowin
2291,f,f,f,f,f,t,f,f,f,f,...,f,f,t,f,f,t,f,t,t,won


## 1.1. Data Dimensions

In [3]:
df.shape

(3195, 37)

## 1.2. Data Types

In [5]:
df.dtypes

f       object
f.1     object
f.2     object
f.3     object
f.4     object
f.5     object
f.6     object
f.7     object
f.8     object
f.9     object
f.10    object
f.11    object
l       object
f.12    object
n       object
f.13    object
f.14    object
t       object
f.15    object
f.16    object
f.17    object
f.18    object
f.19    object
f.20    object
f.21    object
t.1     object
f.22    object
f.23    object
f.24    object
f.25    object
f.26    object
f.27    object
f.28    object
t.2     object
t.3     object
n.1     object
won     object
dtype: object

## 1.3. Check NA

In [6]:
df.isnull().sum()

f       0
f.1     0
f.2     0
f.3     0
f.4     0
f.5     0
f.6     0
f.7     0
f.8     0
f.9     0
f.10    0
f.11    0
l       0
f.12    0
n       0
f.13    0
f.14    0
t       0
f.15    0
f.16    0
f.17    0
f.18    0
f.19    0
f.20    0
f.21    0
t.1     0
f.22    0
f.23    0
f.24    0
f.25    0
f.26    0
f.27    0
f.28    0
t.2     0
t.3     0
n.1     0
won     0
dtype: int64

# 2.0. Data Preparation

## 2.1. Encoding

In [31]:
# Categorical encoding
for i in df:
    df[i] = LabelEncoder().fit_transform( df[i] )

## 2.2. Split dataframe into training and test dataset

In [32]:
X_train, X_test, y_train, y_test = train_test_split( df.iloc[:, :-1], df.iloc[:, -1] )

# 3.0. Machine Learning Modelling

## 3.1. Support Vector Machine

In [35]:
# Parameters and number of iterations
param = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'tol': [0.0001, 0.001, 0.01, 0.1],
    'max_iter': [-1, 200, 500, 1000]
}

MAX_EVAL = 3

### 3.1.1. k = 5 for cross validation

In [47]:
svm_model( X_train, X_test, y_train, y_test, 5, MAX_EVAL, param )

,Regularization,Gamma,Tol,Max_iter,Hold_out,Cross_validation
0,1.0,0.100,0.100,500,0.96,0.96
1,0.1,0.010,0.010,200,0.70,0.64
2,1.0,0.001,0.001,-1,0.86,0.80


### 3.1.2. k = 10 for cross validation

In [49]:
svm_model( X_train, X_test, y_train, y_test, 10, MAX_EVAL, param )

,Regularization,Gamma,Tol,Max_iter,Hold_out,Cross_validation
0,100,1.000,0.001,200,0.98,0.98
1,10,0.001,0.001,200,0.68,0.69
2,1,0.001,0.010,-1,0.86,0.81


## 3.2. Multilayer Perceptron

In [53]:
# Parameters and number of iterations
param = {
    'hls': [ (50,), (100,), (100, 50,), (150, 100,) ],
    'activation': [ 'logistic', 'relu', 'tanh', 'identity' ],
    'solver': [ 'sgd', 'lbfgs', 'adam' ],
}

MAX_EVAL = 3

### 3.2.1. k = 5 for cross validation

In [56]:
mlp_model( X_train, X_test, y_train, y_test, 5, MAX_EVAL, param )

,Hidden_layer_sizes,Activation,Solver,Hold_out,Cross_validation
0,"(50,)",tanh,sgd,0.94,0.93
1,"(100, 50)",logistic,lbfgs,0.99,0.99
2,"(50,)",identity,adam,0.98,0.97


### 3.2.2. k = 10 for cross validation

In [57]:
mlp_model( X_train, X_test, y_train, y_test, 10, MAX_EVAL, param )

,Hidden_layer_sizes,Activation,Solver,Hold_out,Cross_validation
0,"(150, 100)",logistic,sgd,0.53,0.52
1,"(50,)",identity,sgd,0.95,0.94
2,"(150, 100)",logistic,sgd,0.53,0.52


## 3.3. Liner Discriminant Analysis

In [64]:
# Parameters and number of iterations
param = { 'solver': ['svd', 'lsqr', 'eigen'], 'shrinkage': np.arange(0, 1, 0.01) }

MAX_EVAL = 3

### 3.3.1. k = 5 for cross validation

In [73]:
lda_model( X_train, X_test, y_train, y_test, 5, MAX_EVAL, param )

,Solver,Hold_out,Cross_validation
0,svd,0.94,0.94
1,svd,0.94,0.94
2,lsqr,0.94,0.94


### 3.3.2. k = 10 for cross validation

In [68]:
lda_model( X_train, X_test, y_train, y_test, 10, MAX_EVAL, param )

,Solver,Hold_out,Cross_validation
0,lsqr,0.94,0.94
1,svd,0.94,0.94
2,lsqr,0.94,0.94
